## Extraer  dataset


In [ ]:
from google.colab import files
import zipfile
import os

zip_file_path = 'inferencia_np.zip'
extract_dir = '/content/arrays_np'

# Ensure the extraction directory exists
os.makedirs(extract_dir, exist_ok=True)

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        print(f"Opening zip file: {zip_file_path}")
        file_list = zip_ref.namelist()
        print(f"Found {len(file_list)} members in the zip file.")

        for member in file_list:
            try:
                # Extract each member individually
                zip_ref.extract(member, extract_dir)
                print(f"Successfully extracted: {member}")
            except Exception as e:
                print(f"❌ Error extracting {member}: {e}")
                # You can add more specific error handling here if needed
                # For example, check if it's a permission error or a data error

except FileNotFoundError:
    print(f"❌ Error: Zip file not found at {zip_file_path}")
except zipfile.BadZipFile:
    print(f"❌ Error: The file {zip_file_path} is not a valid zip file or is corrupted.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

Opening zip file: inferencia_np.zip
Found 12 members in the zip file.
Successfully extracted: inferencia_np/
Successfully extracted: inferencia_np/inferencia.npy
Successfully extracted: inferencia_np/inferencia_10_0009.npy
Successfully extracted: inferencia_np/inferencia_1_0000.npy
Successfully extracted: inferencia_np/inferencia_2_0001.npy
Successfully extracted: inferencia_np/inferencia_3_0002.npy
Successfully extracted: inferencia_np/inferencia_4_0003.npy
Successfully extracted: inferencia_np/inferencia_5_0004.npy
Successfully extracted: inferencia_np/inferencia_6_0005.npy
Successfully extracted: inferencia_np/inferencia_7_0006.npy
Successfully extracted: inferencia_np/inferencia_8_0007.npy
Successfully extracted: inferencia_np/inferencia_9_0008.npy


# Inferencia


In [ ]:
import numpy as np
import tensorflow as tf
import os

# 📍 Ruta al modelo y al directorio de prueba
modelo_path = '/content/modelo.h5'
test_dir = '/content/arrays_np/inferencia_np'

# 📦 Cargar modelo entrenado
model = tf.keras.models.load_model(modelo_path)

# 🔎 Buscar archivos que comiencen por "grabacion" y terminen en .npy
test_files = [f for f in os.listdir(test_dir)]

print(f"🔍 {len(test_files)} archivos encontrados para inferencia.")

# 📊 Contadores
aciertos = 0
total = 0

# 🧪 Procesar cada archivo
for file in test_files:
    path = os.path.join(test_dir, file)
    try:
        X = np.load(path)

        if X.shape != (30, 128, 128, 1):
            print(f"⚠️ {file} tiene forma incorrecta: {X.shape}")
            continue

        # 🧼 Preprocesamiento
        X = X / 255.0
        X = np.expand_dims(X, axis=0)

        # 🤖 Predicción
        pred = model.predict(X)
        pred_class = np.argmax(pred) + 1

        # 🎯 Clase real desde el nombre del archivo (3er elemento)
        real_class = int(file.split("_")[1])

        # ✔️ Comparar y contar
        correcto = pred_class == real_class
        if correcto:
            aciertos += 1
        total += 1

        print(f"📁 {file} → Predicho: {pred_class} | Real: {real_class} | {'✅ Correcto' if correcto else '❌ Incorrecto'}")

    except Exception as e:
        print(f"❌ Error con {file}: {e}")

# 📈 Mostrar tasa de acierto
if total > 0:
    tasa_acierto = aciertos / total * 100
    print(f"\n🎯 Tasa de acierto: {tasa_acierto:.2f}% ({aciertos}/{total})")
else:
    print("⚠️ No se procesaron archivos correctamente.")


🔍 11 archivos encontrados para inferencia.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 627ms/step
📁 inferencia_9_0008.npy → Predicho: 1 | Real: 9 | ❌ Incorrecto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
📁 inferencia_5_0004.npy → Predicho: 1 | Real: 5 | ❌ Incorrecto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
📁 inferencia_4_0003.npy → Predicho: 4 | Real: 4 | ✅ Correcto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
❌ Error con inferencia.npy: list index out of range
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
📁 inferencia_8_0007.npy → Predicho: 4 | Real: 8 | ❌ Incorrecto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
📁 inferencia_10_0009.npy → Predicho: 10 | Real: 10 | ✅ Correcto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
📁 inferencia_2_0001.npy → Predicho: 4 | Real: 2 | ❌ Incorrecto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
📁 inferencia_7_0006.npy → Predicho: 3 | Real: 7 | ❌ Incorrecto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
📁 inferencia_1_0000.npy → Predicho: 10 | Real: 1 | ❌ Incorrecto
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
📁 inferencia_3_0002.

# Entrenamiento


In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import Sequence

# 🚀 Configuración
data_dir = '/content/arrays_np/arrays7_np'
batch_size = 1
epochs = 30
input_shape = (30, 128, 128, 1)
num_classes = 10

# 🚀 Definir Generador de Datos
class NPYDataGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size=1, num_classes=10, shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_paths = self.file_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        X = []
        for path in batch_paths:
            array = np.load(path)
            if array.shape != (30, 128, 128, 1):
                raise ValueError(f"Archivo con forma incorrecta: {path} → {array.shape}")
            X.append(array / 255.0)

        X = np.array(X)
        y = tf.keras.utils.to_categorical(np.array(batch_labels), num_classes=self.num_classes)

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            temp = list(zip(self.file_paths, self.labels))
            np.random.shuffle(temp)
            self.file_paths, self.labels = zip(*temp)

# 🚀 Cargar rutas y etiquetas
file_paths = []
labels = []

for file in os.listdir(data_dir):
    if file.endswith(".npy"):
        path = os.path.join(data_dir, file)
        file_paths.append(path)
        label = int(file.split("_")[1])
        labels.append(label -1)

file_paths = np.array(file_paths)
labels = np.array(labels)

print("Total archivos:", len(file_paths))

# ✅ Crear carpeta de resultados
os.makedirs("/content/Resultados", exist_ok=True)

# 🚀 Dividir datos en entrenamiento y validación
train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# 🚀 Generadores
train_gen = NPYDataGenerator(train_paths, train_labels, batch_size=batch_size, num_classes=num_classes)
val_gen = NPYDataGenerator(val_paths, val_labels, batch_size=batch_size, num_classes=num_classes, shuffle=False)

# 🚀 Modelo
input_layer = Input(shape=input_shape)
x = TimeDistributed(Conv2D(32, (3,3), activation='relu', padding='same'))(input_layer)
x = TimeDistributed(MaxPooling2D((2,2)))(x)
x = TimeDistributed(Conv2D(64, (3,3), activation='relu', padding='same'))(x)
x = TimeDistributed(MaxPooling2D((2,2)))(x)
x = TimeDistributed(Flatten())(x)
x = LSTM(128)(x)
x = Dropout(0.3)(x)
output_layer = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🚀 Callbacks
checkpoint_path = '/content/Resultados/modelo.h5'
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)
]

# 🚀 Entrenamiento
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

# 🎨 Gráfica de Accuracy y Loss
fig, axs = plt.subplots(1, 2, figsize=(14, 5))

axs[0].plot(history.history['accuracy'], label='Train Accuracy')
axs[0].plot(history.history['val_accuracy'], label='Val Accuracy')
axs[0].set_title('Accuracy')
axs[0].legend()

axs[1].plot(history.history['loss'], label='Train Loss')
axs[1].plot(history.history['val_loss'], label='Val Loss')
axs[1].set_title('Loss')
axs[1].legend()

plt.savefig('/content/Resultados/grafica_entrenamiento.png')
plt.close()

# 🎯 Matriz de Confusión
y_true = []
y_pred = []

for i in range(len(val_gen)):
    X_batch, y_batch = val_gen[i]
    preds = model.predict(X_batch)
    preds_classes = np.argmax(preds, axis=1)
    true_classes = np.argmax(y_batch, axis=1)

    y_true.extend(true_classes)
    y_pred.extend(preds_classes)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title('Matriz de Confusión')
plt.savefig('/content/Resultados/matriz_confusion.png')
plt.close()

report = classification_report(y_true, y_pred, output_dict=True)
with open('/content/Resultados/clasificacion.json', 'w') as f:
    json.dump(report, f, indent=4)

print("\n✅ ¡Entrenamiento completado! Modelo, gráficas, matriz de confusión y reporte guardados.")
